In [1]:
import numpy as np

In [2]:
## Read in the English training file as a big string
train_english_string =''
with open ('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/english/train') as train_english:
    for line in train_english:
        train_english_string += line

In [3]:
## Construct unigram word count for the train string
word_count_unigram = {} ## This contains the word counts for unigram
for cha in train_english_string:
    if cha not in word_count_unigram:
        word_count_unigram[cha]=1
    else:
        word_count_unigram[cha]+=1

In [4]:
## Construct the word string with start word <s>. Using “我” to represent <s>
n = 10 ## This is a 10-gram model
start_word=(n-1)*"我"  ## 我 represent start word <s>
#print (start_word)
train_english_string_full = start_word+train_english_string

In [5]:
### define function to create the tree structure to store the training parameters
def create(ngram, dic):## ngram is a ngram string, dic is a dictionary. 
    n = len(ngram)
    if n == 0: ##base case 
        return
    if ngram[0] not in dic:
        dic[ngram[0]] = []
        dic[ngram[0]].append(1)
        temp_dict = {}
        dic[ngram[0]].append(temp_dict)
    else:
        dic[ngram[0]][0] += 1
    create(ngram[1:],dic[ngram[0]][1])         

In [6]:
## Train the n-gram model
n = 10  ## 10-gram model
ngram_training = {} ## This is tree structure containing the traing results
                    ## Key is the cha, then value is a list; 1st element in the list is the count of the key,
                    ## 2nd element in the list is a dict in which key is the cha left following the previous key
                    ## for n gram, there are n levels of the tree

for i in range(len(train_english_string_full)-(n-1)):
    ngram = ''
    for j in range(n):
        ngram +=  train_english_string_full[i+j]
    create(ngram, ngram_training)

#Deal with the last a few characters 
n = n-1
while n!=0 :
    create(train_english_string_full[-n:],ngram_training)
    n-=1

In [7]:
## define a function to find the counts in the tree structure. This function traverse the tree to find the counts of ngram  
def find_counts (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)                       ## find the counts of ngram
    if n == 1:  ## base case
        if ngram not in ngram_training:
            return 0
        else:
            return ngram_training[ngram][0]    
    else:
        if ngram[0] not in ngram_training:
            return 0
        else:
            return find_counts(ngram[1:],ngram_training[ngram[0]][1])

In [8]:
## define a function to find the subtree 
def find_subtree (ngram, ngram_training): ## ngram_training is a tree structure for n = 10, so len(ngram) should be less than 10
    n = len(ngram)
    if n == 1:
        if ngram not in ngram_training:
            temp ={}
            return temp
        else:
            return ngram_training[ngram][1]    
    else:
        if ngram[0] not in ngram_training:
            temp = {}
            return temp
        else:
            return find_subtree(ngram[1:],ngram_training[ngram[0]][1])

In [10]:
## Define the recursive function for prediction with witten-bell smoothing
## This function returns p(w/u)
def prediction_witten_bell_smoothing(n,w,position_predict,word_string): ## n means n-gram, w is the character to predict, 
                                                                        ## position_predict is the postion to be predicted                                                                       
    ## construct u     
    u=''
    for i in range(n-1,0,-1):
        u += word_string[position_predict-i]   
    
        
    if len(u)==0: ## base case, return p(w)  
        N = len(train_english_string) # number of word token-not including <s>
        n_1_plus = len(word_count_unigram)## number of word type seen at least one time -not including <s>
        n_word_type = len(word_count_unigram)+1 ## number of word type_include unkown word
        lamda = N/(N+n_1_plus)
        return lamda*ngram_training[w][0]/N + (1-lamda)/n_word_type   ## ngram_training[w][0] represents c(w), return p(w)
        
    #if u not in ngram_wordcount[n-1]:
        #return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string)
    if find_counts (u, ngram_training) == 0:
        return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string)
    else:        
        ## calculate c(u_dot) and n1+(u_dot)        
        sub_tree_under_u = find_subtree (u, ngram_training)
        count_u_dot = 0
        for i in sub_tree_under_u:
            count_u_dot+=sub_tree_under_u[i][0]     
        
        count_1plus_u_dot = 0
        for i in sub_tree_under_u:
            count_1plus_u_dot+=1 
            
        if count_u_dot == 0:
            return prediction_witten_bell_smoothing(n-1,w,position_predict,word_string)
                
        ## calculate lamda_u
        lamda_u = count_u_dot/(count_u_dot+count_1plus_u_dot)
        
        ## calculate c(uw)
        c_uw = find_counts(u+w, ngram_training)
        
        return lamda_u*c_uw/count_u_dot+(1-lamda_u)*prediction_witten_bell_smoothing(n-1,w,position_predict,word_string)

In [11]:
## Read in the English dev file as a big string
with open('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/english/dev') as dev_english:
    dev_english_string =''
    for line in dev_english:
        dev_english_string += line

n=10
start_word=(n-1)*"我"
dev_english_string_full = start_word+dev_english_string

In [12]:
## Read in the English test file as a big string
with open('C:/Users/Xin/Desktop/NLP/Homework2/hw2-data/english/test') as test_english:
    test_english_string =''
    for line in test_english:
        test_english_string += line
n=10
start_word=(n-1)*"我"
test_english_string_full = start_word+test_english_string

In [15]:
## Print out the most probable characters and their probabilities for the first ten position in the dev file

n = 10
ngram_prediction = {} ## This stores the most probable characters and their probabilities for the first ten positions

for position in range(9,19): ## The first 10 position in dev file not including the start word
    prob_each_character = {}
    for cha in word_count_unigram:
        prob_each_character[cha]= prediction_witten_bell_smoothing(n,cha,position,dev_english_string_full)
    ngram_prediction[position-8] = [key for key,val in prob_each_character.items() if val == max(prob_each_character.values())]
    ngram_prediction[position-8].append(prob_each_character[ngram_prediction[position-8][0]]) 
    
print (ngram_prediction)
    

{1: ['i', 0.6979471503647389], 2: ['o', 0.1461454742412194], 3: ['n', 0.29419482257739776], 4: ['t', 0.36402328733414724], 5: ['i', 0.620071963982971], 6: ['c', 0.9478949322653932], 7: ['i', 0.9999014440869798], 8: ['u', 0.9999933028641393], 9: ['s', 0.9999997718863204], 10: [':', 0.7368416741212633]}


In [30]:
## n=10, Prediction on dev file
n=10 
ngram_prediction = {}

#count = 0
for position in range(n-1,len(dev_english_string_full)):
    #count += 1
    #if count == 1000:
        #print (count) ## The print out the process of the prediction
        #count = 0
    ngram_prediction_character = {}
    for cha in word_count_unigram:
        ngram_prediction_character[cha]= prediction_witten_bell_smoothing(n,cha,position,dev_english_string_full)
    ngram_prediction[position] = [key for key,val in ngram_prediction_character.items() if val == max(ngram_prediction_character.values())]

count_correct_predict = 0
for position in range(n-1,len(dev_english_string)):
    if dev_english_string_full[position]==ngram_prediction[position][0]:
        count_correct_predict+=1
prediction_accuracy = count_correct_predict/(len(dev_english_string_full)-(n-1))

print("Prediction Accuracy on Dev File:{}".format(prediction_accuracy))


Prediction Accuracy on Dev File:0.6162902914495271


In [29]:
## Test n=10, on test file
n=10 
ngram_prediction = {}

#count = 0
for position in range(n-1,len(test_english_string_full)):
    #count += 1
    #if count == 1000:
        #print (count) 
        #count = 0
    ngram_prediction_character = {}
    for cha in word_count_unigram:
        ngram_prediction_character[cha]= prediction_witten_bell_smoothing(n,cha,position,test_english_string_full)
    ngram_prediction[position] = [key for key,val in ngram_prediction_character.items() if val == max(ngram_prediction_character.values())]

count_correct_predict = 0
for position in range(n-1,len(test_english_string_full)):
    if test_english_string_full[position]==ngram_prediction[position][0]:
        count_correct_predict+=1
prediction_accuracy = count_correct_predict/(len(test_english_string_full)-(n-1))

print("Prediction Accuracy on Test File:{}" .format(prediction_accuracy))


Prediction Accuracy on Test File: 0.6048300261498231


In [15]:
## Calculate the probability of each character in the test file
n=10
Probability_each_character = {}
for position in range(n-1,len(test_english_string_full)):
    Probability_each_character[test_english_string_full[position]]= prediction_witten_bell_smoothing(n,test_english_string_full[position],position,test_english_string_full)

In [28]:
## Calculate 10-gram model's perplexity on the test data set
summation_log_prob = 0

for cha in Probability_each_character:
    summation_log_prob += np.log(Probability_each_character[cha])
    
perplexity = np.exp(-summation_log_prob/len(test_english_string))
print ("Perplexity = {}".format(perplexity))

Perplexity = 1.0467674194667902
